# Instructor Do: Intro to VADER Sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mshel\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [5]:
# Create a newsapi client
newsapi = NewsApiClient(api_key= "22b1baf9e3d84f549a60cd9ca7881995")



In [6]:
# Fetch all the news about Facebook Libra
libra_headlines = newsapi.get_everything(
    q="facebook AND libra",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Facebook Libra: {libra_headlines['totalResults']}")

# Show sample article
libra_headlines["articles"][0]



Total articles about Facebook Libra: 77


{'source': {'id': 'reuters', 'name': 'Reuters'},
 'author': 'Reuters Staff',
 'title': "Big-tech's stablecoins may hurt privacy and innovation: ECB - Reuters",
 'description': "A stablecoin managed by a big tech company, like Facebook's <FB.O> proposed libra, would raise concerns about data protection and even choke financial innovation, European Central Bank board member Fabio Panetta said on Wednesday.",
 'url': 'https://www.reuters.com/article/us-ecb-policy-crypto-idUSKBN27K12V',
 'urlToImage': 'https://static.reuters.com/resources/r/?m=02&d=20201104&t=2&i=1539949013&r=LYNXMPEGA30NS&w=800',
 'publishedAt': '2020-11-04T09:07:00Z',
 'content': 'By Reuters Staff\r\nFILE PHOTO: Shadow of a 3D-printed Facebook Libra cryptocurrency logo is seen near cryptocurrency representation in this illustration taken, September 13, 2019. REUTERS/Dado Ruvic\r\n… [+415 chars]'}

In [7]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()



,date,text,compound,positive,negative,neutral
0,2020-11-04,By Reuters Staff\r\nFILE PHOTO: Shadow of a 3D...,0.0000,0.000,0.0,1.000
1,2020-11-04,By Reuters Staff\r\nFILE PHOTO: Shadow of a 3D...,0.0000,0.000,0.0,1.000
2,2020-11-06,A paper coauthored by scientists at Facebook's...,0.0000,0.000,0.0,1.000
3,2020-11-05,LONDON (Reuters) - Bitcoin's BTC=BTSP price ro...,0.4215,0.080,0.0,0.920
4,2020-11-05,LONDON (Reuters) - Bitcoins price rose to more...,0.4215,0.078,0.0,0.922


In [8]:
# Get descriptive stats from the DataFrame
libra_df.describe()



,compound,positive,negative,neutral
count,77.000000,77.000000,77.000000,77.000000
mean,0.186999,0.064351,0.019870,0.915779
std,0.361103,0.079873,0.044979,0.085036
min,-0.757900,0.000000,0.000000,0.588000
25%,0.000000,0.000000,0.000000,0.888000
50%,0.177900,0.052000,0.000000,0.921000
75%,0.440400,0.093000,0.000000,1.000000
max,0.953100,0.412000,0.195000,1.000000
